# Problem A: Obsługa żądania DELETE w celu usunięcia książki z koszyka

## Prompt i modyfikowany kod

In [ ]:
@RestController
@RequestMapping("/carts")
public class CartController {
 private final CartService cartService;
 public CartController(CartService cartService) {
 this.cartService = cartService;
 }
 @GetMapping
 public Cart getCartContent() {
 return cartService.getCart();
 }
 @PostMapping("/{isbnNumber}")
 public ResponseEntity<Cart> addBookToCart(
 @PathVariable String isbnNumber) {
 return cartService.addBookToCart(isbnNumber);
 }

@Service
@SessionScope
public class CartService {
 private final BookRepository bookRepository;
 private Cart cart;
 public CartService(BookRepository bookRepository) {
 this.bookRepository = bookRepository;
 this.cart = new Cart(Date.from(Instant.now()),
 Date.from(Instant.now()), new ArrayList<>());
 }
 public Cart getCart() {
 return cart;
 }
 public ResponseEntity<Cart> addBookToCart(String isbn) {
 Mono<Book> bookMono = bookRepository.findByIsbnNumber(isbn);
 Optional<Book> book = bookMono.blockOptional();
 if (book.isPresent()) {
 cart.getBookList().add(book.get());
 cart.setUpdatedAt(Date.from(Instant.now()));
 return ResponseEntity.ok(cart);
 }
 return ResponseEntity.notFound().build();
 }
 public void clearCart() {
 this.cart = new Cart(Date.from(Instant.now()),
 Date.from(Instant.now()), new ArrayList<>());
 }
}

public interface BookRepository 
extends ReactiveMongoRepository<Book, String> {
 Flux<Book> findAll();
 Flux<Book> findByIsbnNumber(String isbnNumber);
 Flux<Book> findAllByAuthorContainingAndTitleContaining(
 String author, String title);
}

@Getter
@Setter
@AllArgsConstructor
@Document("books")
public class Book {
 @Id
 private String id;
 private String isbnNumber;
 private String title;
 private String author;
 private Date releaseDate;
}


Add endpoint to this application that allows user to delete book from cart, endpoint should be 
named: /carts/{isbnNumber}, where isbnNumber is property of book that user wants to delete 
from cart, return value should be equal to 200 or 204 if book is successfully deleted from cart 
and 404 if it was not found in cart.


## Odpowiedzi

### Llama

In [ ]:
To add an endpoint to the application to allow users to delete a book from their cart, you can create a new method in the CartController class. The method will be named deleteBookFromCart, and it should take the ISBN number of the book as a parameter.

Here is the code for the new endpoint:

java
 
 

@DeleteMapping("/{isbnNumber}")
public ResponseEntity<Cart> deleteBookFromCart(@PathVariable String isbnNumber) {
    return cartService.deleteBookFromCart(isbnNumber);
}

To handle this request in the CartService class, you can add a new method called deleteBookFromCart. This method will find the book with the specified ISBN number in the cart and remove it if it exists. Here is the code for the new method:

java
 
 

public ResponseEntity<Cart> deleteBookFromCart(String isbnNumber) {
    Optional<Book> book = cart.getBookList().stream()
            .filter(b -> b.getIsbnNumber().equals(isbnNumber))
            .findFirst();

    if (book.isPresent()) {
        cart.getBookList().remove(book.get());
        cart.setUpdatedAt(Date.from(Instant.now()));
        return ResponseEntity.ok(cart);
    }

    return ResponseEntity.notFound().build();
}

In this code, we first find the book with the specified ISBN number in the cart using a stream operation. If the book is found, we remove it from the cart and update the cart's last updated date. Finally, we return the updated cart.

If the book is not found in the cart (i.e., the user did not have that book in their cart), we return a 404 Not Found response.

In [ ]:
To add an endpoint to your application that allows users to delete a book from their cart, you can create a new method in the CartController class and annotate it with @DeleteMapping. This will handle HTTP DELETE requests.

Here's how you can do it:

java
 
 

@DeleteMapping("/{isbnNumber}")
public ResponseEntity<Cart> removeBookFromCart(@PathVariable String isbnNumber) {
    return cartService.removeBookFromCart(isbnNumber);
}

Then, in the CartService class, you need to add a new method that will handle this request. This method should find the book with the given ISBN number in the cart and remove it if it's present:

java
 
 

public ResponseEntity<Cart> removeBookFromCart(String isbnNumber) {
    Optional<Book> book = bookRepository.findByIsbnNumber(isbnNumber);
    
    if (book.isPresent()) {
        Cart cart = this.cart;
        cart.getBookList().remove(book.get());
        cart.setUpdatedAt(Date.from(Instant.now()));
        return ResponseEntity.ok(cart);
    } else {
        return ResponseEntity.notFound().build();
    }
}

In this method, findByIsbnNumber is a method that you need to add in your BookRepository. This method should return an Optional<Book> containing the book with the given ISBN number. If no such book exists, it should return an empty Optional.

Here's how you can do it:

java
 
 

public interface BookRepository extends MongoRepository<Book, String> {
    Optional<Book> findByIsbnNumber(String isbnNumber);
}

Now, when a user sends a DELETE request to /carts/{isbnNumber}, the book with that ISBN number will be removed from their cart.

In [ ]:
To add an endpoint to your application that allows users to delete a book from their cart, you can create a new method in the CartController class and annotate it with @DeleteMapping. This will handle HTTP DELETE requests.

Here's how you can do it:

java
 
 

@DeleteMapping("/{isbnNumber}")
public ResponseEntity<Cart> removeBookFromCart(@PathVariable String isbnNumber) {
    Cart cart = cartService.getCart();
    
    // Check if the book exists in the cart
    for (Book book : cart.getBookList()) {
        if (book.getIsbnNumber().equals(isbnNumber)) {
            cart.getBookList().remove(book);
            return ResponseEntity.ok(cart); // Return the updated cart
        }
    }
    
    // If the book is not found in the cart, return a 404 error
    return ResponseEntity.notFound().build();
}

This method takes an isbnNumber as a path variable and uses it to find the book in the cart. It then removes that book from the cart and returns the updated cart.

You'll also need to add this endpoint to your controller's mapping:

java
 
 

@GetMapping
public Cart getCartContent() {
    return cartService.getCart();
}

@PostMapping("/{isbnNumber}")
public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
    // ...
}

@DeleteMapping("/{isbnNumber}")
public ResponseEntity<Cart> removeBookFromCart(@PathVariable String isbnNumber) {
    // ...
}

Now, when a user sends a DELETE request to /carts/{isbnNumber}, the book with that ISBN number will be removed from their cart.

In [ ]:
To add an endpoint for deleting a book from the cart, you can create a new method in your CartController class and annotate it with @DeleteMapping. This will handle HTTP DELETE requests.

Here's how you can do it:

java
 
 

@DeleteMapping("/{isbnNumber}")
public ResponseEntity<Cart> removeBookFromCart(@PathVariable String isbnNumber) {
    Cart cart = cartService.getCart();

    // Check if the book exists in the cart
    for (Book book : cart.getBookList()) {
        if (book.getIsbnNumber().equals(isbnNumber)) {
            cart.getBookList().remove(book);
            return ResponseEntity.ok(cart);  // Return the updated cart
        }
    }

    // If the book is not found in the cart, return a 404 error
    return ResponseEntity.notFound().build();
}

This method takes an isbnNumber as a path variable and uses it to find the book in the cart. It then removes that book from the cart and returns the updated cart.

You'll also need to add this endpoint to your controller's mapping:

java
 
 

@RestController
@RequestMapping("/carts")
public class CartController {

    private final CartService cartService;

    public CartController(CartService cartService) {
        this.cartService = cartService;
    }

    @GetMapping
    public Cart getCartContent() {
        return cartService.getCart();
    }

    @PostMapping("/{isbnNumber}")
    public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
        // ...
    }

    @DeleteMapping("/{isbnNumber}")
    public ResponseEntity<Cart> removeBookFromCart(@PathVariable String isbnNumber) {
        // ...
    }
}

Now, when a user sends a DELETE request to /carts/{isbnNumber}, the book with that ISBN number will be removed from their cart.

### CodeLlama

In [ ]:
You can add an endpoint to the CartController class to delete a book from the cart. Here's an example implementation:

 
 

@DeleteMapping("/{isbnNumber}")
public ResponseEntity<Cart> removeBookFromCart(@PathVariable String isbnNumber) {
    Optional<Book> book = bookRepository.findById(isbnNumber);
    if (book.isPresent()) {
        cartService.removeBookFromCart(book.get());
        return ResponseEntity.ok(cartService.getCart());
    } else {
        return ResponseEntity.notFound().build();
    }
}

This endpoint will remove the book with the specified isbnNumber from the cart and return the updated cart in the response body. If the book is not found, it will return a 404 Not Found response.

You can also use @RequestBody to delete multiple books at once by sending an array of isbnNumbers in the request body. Here's an example implementation:

 
 

@DeleteMapping("/")
public ResponseEntity<Cart> removeBooksFromCart(@RequestBody List<String> isbnNumbers) {
    for (String isbnNumber : isbnNumbers) {
        Optional<Book> book = bookRepository.findById(isbnNumber);
        if (book.isPresent()) {
            cartService.removeBookFromCart(book.get());
        }
    }
    return ResponseEntity.ok(cartService.getCart());
}

This endpoint will remove all books with the specified isbnNumbers from the cart and return the updated cart in the response body. If any of the books are not found, they will be ignored.

In [ ]:
To add an endpoint to the application to allow users to delete a book from their cart, you can follow these steps:


    Add a new method in the CartService class to handle the deletion of a book from the cart. This method should take the ISBN number of the book as a parameter and remove it from the user's cart.


 
 

public void deleteBookFromCart(String isbnNumber) {
    cart.getBookList().removeIf(book -> book.getIsbnNumber().equals(isbnNumber));
}


    Add a new endpoint to the CartController class that maps to the new method you just created. This endpoint should have a path variable for the ISBN number of the book to be deleted.


 
 

@DeleteMapping("/{isbnNumber}")
public ResponseEntity<Cart> deleteBookFromCart(@PathVariable String isbnNumber) {
    cartService.deleteBookFromCart(isbnNumber);
    return ResponseEntity.ok(cartService.getCart());
}


    Update the addBookToCart method in the CartController class to check if the book already exists in the user's cart before adding it again. This will prevent duplicate books from being added to the cart.


 
 

@PostMapping("/{isbnNumber}")
public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
    Optional<Book> book = bookRepository.findById(isbnNumber).blockOptional();
    if (book.isPresent()) {
        cartService.addBookToCart(isbnNumber);
        return ResponseEntity.ok(cartService.getCart());
    } else {
        return ResponseEntity.notFound().build();
    }
}

With these changes, the application will now have an endpoint to delete a book from the user's cart by specifying its ISBN number.

In [ ]:
Here's an example implementation for the /carts/{isbnNumber} endpoint:

java
 
 

@RestController
public class CartController {
    // ...
    
    @DeleteMapping("/{isbnNumber}")
    public ResponseEntity<Cart> deleteBookFromCart(@PathVariable String isbnNumber) {
        Optional<Book> book = cartService.getCart().getBookList().stream()
                .filter(b -> b.getIsbnNumber().equals(isbnNumber))
                .findAny();
        
        if (book.isPresent()) {
            cartService.getCart().getBookList().remove(book.get());
            return ResponseEntity.ok(cartService.getCart());
        }
        
        return ResponseEntity.notFound().build();
    }
}

This endpoint will remove the book with the specified isbnNumber from the user's cart if it exists. The deleteBookFromCart() method in the CartService class will also need to be updated to reflect this change:

java
 
 

@Service
public class CartService {
    // ...
    
    public void deleteBookFromCart(String isbnNumber) {
        cart.getBookList().removeIf(b -> b.getIsbnNumber().equals(isbnNumber));
    }
}

This will ensure that the book with the specified isbnNumber is removed from the user's cart when the /carts/{isbnNumber} endpoint is called.

In [ ]:
Here's an example implementation for the /carts/{isbnNumber} endpoint:

java
 
 

@RestController
public class CartController {
    // ...
    
    @DeleteMapping("/{isbnNumber}")
    public ResponseEntity<Cart> deleteBookFromCart(@PathVariable String isbnNumber) {
        cartService.deleteBookFromCart(isbnNumber);
        return ResponseEntity.ok().body(cartService.getCart());
    }
}

This endpoint will allow the user to delete a book from their cart by specifying its isbnNumber. The deleteBookFromCart() method in the CartService class will also need to be updated to reflect this change:

java
 
 

@Service
public class CartService {
    // ...
    
    public void deleteBookFromCart(String isbnNumber) {
        cart.getBookList().removeIf(book -> book.getIsbnNumber().equals(isbnNumber));
    }
}

This will ensure that the book with the specified isbnNumber is removed from the user's cart when the /carts/{isbnNumber} endpoint is called.[INST:

In [ ]:
Continuing from where we left off, let's add the /carts/{isbnNumber} endpoint to our web application. This endpoint will allow users to delete books from their cart by specifying the isbnNumber of the book they want to remove.

Here's an example implementation for the /carts/{isbnNumber} endpoint:

java
 
 

@RestController
public class CartController {
    // ...
    
    @DeleteMapping("/{isbnNumber}")
    public ResponseEntity<Cart> deleteBookFromCart(@PathVariable String isbnNumber) {
        cartService.deleteBookFromCart(isbnNumber);
        return ResponseEntity.ok().body(cartService.getCart());
    }
}

This endpoint will allow the user to delete a book from their cart by specifying its isbnNumber. The deleteBookFromCart() method in the CartService class will also need to be updated to reflect this change:

java
 
 

@Service
public class CartService {
    // ...
    
    public void deleteBookFromCart(String isbnNumber) {
        cart.getBookList().removeIf(book -> book.getIsbnNumber().equals(isbnNumber));
    }
}

This will ensure that the book with the specified isbnNumber is removed from the user's cart when the /carts/{isbnNumber} endpoint is called.

Next, we need to add a method in our CartService class to handle the deletion of books from the cart. We can use the removeIf() method on the list of books in the cart to remove any book that has the specified isbnNumber. Here's an example implementation:

java
 
 

@Service
public class CartService {
    // ...
    
    public void deleteBookFromCart(String isbnNumber) {
        cart.getBookList().removeIf(book -> book.getIsbnNumber().equals(isbnNumber));
    }
}

This method will remove any book from the user's cart that has the specified isbnNumber. We can then use this method in our /carts/{isbnNumber} endpoint to delete a book from the cart.

Finally, we need to update our /carts endpoint to handle the deletion of books from the cart. We can do this by adding a new method to our CartController class that handles the deletion of books:

java
 
 

@RestController
public class CartController {
    // ...
    
    @DeleteMapping("/{isbnNumber}")
    public ResponseEntity<Cart> deleteBookFromCart(@PathVariable String isbnNumber) {
        cartService.deleteBookFromCart(isbnNumber);
        return ResponseEntity.ok().body(cartService.getCart());
    }
}

This method will call the deleteBookFromCart() method in our CartService class to remove any book from the user's cart that has the specified isbnNumber. It will then return a response entity with the updated cart information.

With this implementation, users can now delete books from their cart by specifying the isbnNumber of the book they want to remove. This allows us to provide a more convenient and user-friendly experience for our web application.

### Dolphin-Mixtral

In [ ]:
To add an endpoint to the web app for deleting a book from the cart using the /carts/{isbnNumber} URL path, you can extend the existing CartController by adding a new DeleteBookFromCart method. Here's how it could be done:

java
 
 

@RestController
@RequestMapping("/carts")
public class CartController {

    private final CartService cartService;

    public CartController(CartService cartService) {
        this.cartService = cartService;
    }

    @GetMapping
    public Cart getCartContent() {
        return cartService.getCart();
    }

    @PostMapping("/{isbnNumber}")
    public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
        return cartService.addBookToCart(isbnNumber);
    }

    @DeleteMapping("/{isbnNumber}")
    public ResponseEntity<String> deleteBookFromCart(@PathVariable String isbnNumber) {
        return cartService.deleteBookFromCart(isbnNumber);
    }
}

And then you need to implement the deleteBookFromCart method in the CartService class:

java
 
 

@Service
@SessionScope
public class CartService {

    private final BookRepository bookRepository;
    private Cart cart;

    public CartService(BookRepository bookRepository) {
        this.bookRepository = bookRepository;
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }

    public Cart getCart() {
        return cart;
    }

    public ResponseEntity<Cart> addBookToCart(String id) {
        Mono<Book> bookMono = bookRepository.findById(id);
        Optional<Book> book = bookMono.blockOptional();

        if (book.isPresent()) {
            cart.getBookList().add(book.get());
            cart.setUpdatedAt(Date.from(Instant.now()));
            return ResponseEntity.ok(cart);
        }

        return ResponseEntity.notFound().build();
    }

    public void clearCart() {
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }

    public ResponseEntity<String> deleteBookFromCart(String isbnNumber) {
        Optional<Book> bookOptional = bookRepository.findByIsbnNumber(isbnNumber);

        if (bookOptional.isPresent()) {
            Book book = bookOptional.get();
            cart.getBookList().removeIf(b -> b.getId().equals(book.getId()));
            return ResponseEntity.ok("Book deleted from cart.");
        }

        return ResponseEntity.notFound().build();
    }
}

In this updated implementation, the deleteBookFromCart method checks if a book with the given isbnNumber exists in the repository. If it does, it removes the corresponding book from the cart and returns a success message. Otherwise, it returns a not found response.

Remember to also add the appropriate dependencies for Spring MVC, Spring Web, and Spring Data MongoDB if you haven't already done so.

In [ ]:
To add an endpoint for deleting books from the cart, you can use the DELETE HTTP method and specify the isbnNumber as a path parameter. Here's how you can modify your CartController class:

java
 
 

@RestController
@RequestMapping("/carts")
public class CartController {

    private final CartService cartService;

    public CartController(CartService cartService) {
        this.cartService = cartService;
    }

    @GetMapping
    public Cart getCartContent() {
        return cartService.getCart();
    }

    @PostMapping("/{isbnNumber}")
    public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
        return cartService.addBookToCart(isbnNumber);
    }

    @DeleteMapping("/{isbnNumber}")
    public ResponseEntity<String> deleteBookFromCart(@PathVariable String isbnNumber) {
        return cartService.deleteBookFromCart(isbnNumber);
    }
}

In the CartService class, you need to implement the deleteBookFromCart() method to remove a book from the cart based on its ISBN number:

java
 
 

@Service
@SessionScope
public class CartService {

    private final BookRepository bookRepository;
    private Cart cart;

    public CartService(BookRepository bookRepository) {
        this.bookRepository = bookRepository;
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }

    public Cart getCart() {
        return cart;
    }

    public ResponseEntity<Cart> addBookToCart(String id) {
        Mono<Book> bookMono = bookRepository.findById(id);
        Optional<Book> book = bookMono.blockOptional();

        if (book.isPresent()) {
            cart.getBookList().add(book.get());
            cart.setUpdatedAt(Date.from(Instant.now()));
            return ResponseEntity.ok(cart);
        }

        return ResponseEntity.notFound().build();
    }

    public ResponseEntity<String> deleteBookFromCart(String isbnNumber) {
        for (Book book : cart.getBookList()) {
            if (book.getIsbnNumber().equalsIgnoreCase(isbnNumber)) {
                cart.getBookList().remove(book);
                break;
            }
        }
        cart.setUpdatedAt(Date.from(Instant.now()));
        return ResponseEntity.ok("Book deleted successfully.");
    }
}

This allows users to add books to their cart by making a POST request with the ISBN number as a path parameter, and they can delete a book from their cart by making a DELETE request with the ISBN number as a path parameter.

In [ ]:
To add an endpoint for deleting a book from the cart, you can modify the CartController class as follows:

java
 
 

@RestController
@RequestMapping("/carts")
public class CartController {

    private final CartService cartService;

    public CartController(CartService cartService) {
        this.cartService = cartService;
    }

    @GetMapping
    public Cart getCartContent() {
        return cartService.getCart();
    }

    @PostMapping("/{isbnNumber}")
    public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
        return cartService.addBookToCart(isbnNumber);
    }

    @DeleteMapping("/{isbnNumber}")
    public ResponseEntity<Cart> deleteBookFromCart(@PathVariable String isbnNumber) {
        return cartService.deleteBookFromCart(isbnNumber);
    }

}

Then, modify the CartService class as follows:

java
 
 

@Service
@SessionScope
public class CartService {

    private final BookRepository bookRepository;
    private Cart cart;

    public CartService(BookRepository bookRepository) {
        this.bookRepository = bookRepository;
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }

    public Cart getCart() {
        return cart;
    }

    public ResponseEntity<Cart> addBookToCart(String id) {
        Mono<Book> bookMono = bookRepository.findById(id);
        Optional<Book> book = bookMono.blockOptional();

        if (book.isPresent()) {
            cart.getBookList().add(book.get());
            cart.setUpdatedAt(Date.from(Instant.now()));
            return ResponseEntity.ok(cart);
        }

        return ResponseEntity.notFound().build();
    }

    public ResponseEntity<Cart> deleteBookFromCart(String id) {
        Optional<Book> book = cart.getBookList().stream()
                .filter(b -> b.getId().equals(id))
                .findFirst();

        if (book.isPresent()) {
            cart.getBookList().removeIf(b -> b.getId().equals(id));
            cart.setUpdatedAt(Date.from(Instant.now()));
            return ResponseEntity.ok(cart);
        }

        return ResponseEntity.notFound().build();
    }

    public void clearCart() {
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }
}

With these modifications, you can now send a DELETE request to the endpoint "/carts/{isbnNumber}" to remove the book with the specified ISBN number from the cart. The {isbnNumber} will be replaced by the actual ISBN number value in your request URL.

In [ ]:
To add an endpoint for the user to delete a book from their cart in this Spring Boot application, you would need to modify your CartController.java and CartService.java as follows:


    Modify CartController.java:


java
 
 

import org.springframework.http.ResponseEntity;
import org.springframework.web.bind.annotation.*;

@RestController
@RequestMapping("/carts")
public class CartController {

    private final CartService cartService;

    public CartController(CartService cartService) {
        this.cartService = cartService;
    }

    @GetMapping
    public Cart getCartContent() {
        return cartService.getCart();
    }

    @PostMapping("/{isbnNumber}")
    public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
        return cartService.addBookToCart(isbnNumber);
    }

    @DeleteMapping("/{isbnNumber}")
    public ResponseEntity<String> deleteBookFromCart(@PathVariable String isbnNumber) {
        return cartService.deleteBookFromCart(isbnNumber);
    }
}


    Modify CartService.java:


java
 
 

import org.springframework.http.ResponseEntity;
import reactor.core.publisher.Mono;

@Service
@SessionScope
public class CartService {

    private final BookRepository bookRepository;
    private Cart cart;

    public CartService(BookRepository bookRepository) {
        this.bookRepository = bookRepository;
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }

    public Cart getCart() {
        return cart;
    }

    public ResponseEntity<Cart> addBookToCart(String id) {
        Mono<Book> bookMono = bookRepository.findById(id);
        Optional<Book> book = bookMono.blockOptional();

        if (book.isPresent()) {
            cart.getBookList().add(book.get());
            cart.setUpdatedAt(Date.from(Instant.now()));
            return ResponseEntity.ok(cart);
        }

        return ResponseEntity.notFound().build();
    }

    public void clearCart() {
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }

    public Mono<ResponseEntity<String>> deleteBookFromCart(String isbnNumber) {
        return cart.getBookList().stream()
                .filter(book -> book.getIsbnNumber().equals(isbnNumber))
                .findFirst()
                .map(book -> {
                    this.cart.getBookList().removeIf(b -> b.getIsbnNumber().equals(isbnNumber));
                    return ResponseEntity.ok("Book deleted from cart successfully.");
                })
                .orElseGet(() -> Mono.just(ResponseEntity.notFound().build()));
    }
}

This modification adds a @DeleteMapping to the CartController and implements the corresponding delete method in the CartService. The delete method uses a stream operation to filter books by their ISBN number, removes them from the cart list if found, and returns an appropriate response entity depending on whether or not a book matching the ISBN number was found in the cart.

You will also need to add this endpoint to your application's configuration (e.g., application.properties or application.yml) file:

yaml
 
 

spring.mvc.servlet.path=/
spring.mvc.add-interceptors=false
spring.datasource.url=...
spring.jpa.show-sql=true
spring.jpa.properties.hibernate.ddl-auto=update
server.port=8080

This configuration sets the context path of your Spring Boot application to /. If you do not specify a context path, it defaults to /api, which would result in your endpoints being accessible at /api/carts/{isbnNumber} instead of /carts/{isbnNumber}.

### ChatGPT

In [ ]:
To implement the functionality to delete a book from the cart, you can modify the CartController to include a @DeleteMapping method. Here's how you can do it:

java

@RestController
@RequestMapping("/carts")
public class CartController {

    private final CartService cartService;

    public CartController(CartService cartService) {
        this.cartService = cartService;
    }

    @GetMapping
    public Cart getCartContent() {
        return cartService.getCart();
    }

    @PostMapping("/{isbnNumber}")
    public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
        return cartService.addBookToCart(isbnNumber);
    }

    @DeleteMapping("/{isbnNumber}")
    public ResponseEntity<Cart> removeBookFromCart(@PathVariable String isbnNumber) {
        return cartService.removeBookFromCart(isbnNumber);
    }

}

Then, in the CartService class, add the method removeBookFromCart:

java

@Service
@SessionScope
public class CartService {

    private final BookRepository bookRepository;
    private Cart cart;

    public CartService(BookRepository bookRepository) {
        this.bookRepository = bookRepository;
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }

    public Cart getCart() {
        return cart;
    }

    public ResponseEntity<Cart> addBookToCart(String id) {
        Mono<Book> bookMono = bookRepository.findById(id);
        Optional<Book> book = bookMono.blockOptional();

        if (book.isPresent()) {
            cart.getBookList().add(book.get());
            cart.setUpdatedAt(Date.from(Instant.now()));
            return ResponseEntity.ok(cart);
        }

        return ResponseEntity.notFound().build();
    }

    public ResponseEntity<Cart> removeBookFromCart(String isbnNumber) {
        Optional<Book> bookToRemove = cart.getBookList().stream()
                .filter(book -> book.getIsbnNumber().equals(isbnNumber))
                .findFirst();

        if (bookToRemove.isPresent()) {
            cart.getBookList().remove(bookToRemove.get());
            cart.setUpdatedAt(Date.from(Instant.now()));
            return ResponseEntity.ok(cart);
        }

        return ResponseEntity.notFound().build();
    }

    public void clearCart() {
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }
}

With this setup, when a DELETE request is made to /carts/{isbnNumber}, where {isbnNumber} is the ISBN number of the book to be removed from the cart, it will remove that book from the cart if it exists. If the book is not found in the cart, it will return a 404 Not Found response.

In [ ]:
To add an endpoint for deleting a book from the cart using the ISBN number, you can modify the CartController to include a @DeleteMapping method. Here's how you can do it:

java

@RestController
@RequestMapping("/carts")
public class CartController {

    private final CartService cartService;

    public CartController(CartService cartService) {
        this.cartService = cartService;
    }

    @GetMapping
    public Cart getCartContent() {
        return cartService.getCart();
    }

    @PostMapping("/{isbnNumber}")
    public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
        return cartService.addBookToCart(isbnNumber);
    }

    @DeleteMapping("/{isbnNumber}")
    public ResponseEntity<Cart> removeBookFromCart(@PathVariable String isbnNumber) {
        return cartService.removeBookFromCart(isbnNumber);
    }

}

Then, in the CartService class, add the method removeBookFromCart:

java

@Service
@SessionScope
public class CartService {

    private final BookRepository bookRepository;
    private Cart cart;

    public CartService(BookRepository bookRepository) {
        this.bookRepository = bookRepository;
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }

    public Cart getCart() {
        return cart;
    }

    public ResponseEntity<Cart> addBookToCart(String id) {
        Mono<Book> bookMono = bookRepository.findById(id);
        Optional<Book> book = bookMono.blockOptional();

        if (book.isPresent()) {
            cart.getBookList().add(book.get());
            cart.setUpdatedAt(Date.from(Instant.now()));
            return ResponseEntity.ok(cart);
        }

        return ResponseEntity.notFound().build();
    }

public ResponseEntity<Cart> removeBookFromCart(String isbnNumber) {
    Optional<Book> bookToRemove = cart.getBookList().stream()
            .filter(book -> book.getIsbnNumber().equals(isbnNumber))
            .findFirst();

    if (bookToRemove.isPresent()) {
        cart.getBookList().remove(bookToRemove.get());
        cart.setUpdatedAt(Date.from(Instant.now()));
        return ResponseEntity.ok(cart);
    }

    return ResponseEntity.notFound().build();
}

    public void clearCart() {
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }
}

With this setup, when a DELETE request is made to /carts/{isbnNumber}, where {isbnNumber} is the ISBN number of the book to be removed from the cart, it will remove that book from the cart if it exists. If the book is not found in the cart, it will return a 404 Not Found response.


In [ ]:
To add an endpoint for deleting a book from the cart in your Spring Boot application, you can modify the CartController class to include a method for handling the deletion. Here's how you can do it:

java

@RestController
@RequestMapping("/carts")
public class CartController {

    private final CartService cartService;

    public CartController(CartService cartService) {
        this.cartService = cartService;
    }

    @GetMapping
    public Cart getCartContent() {
        return cartService.getCart();
    }

    @PostMapping("/{isbnNumber}")
    public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
        return cartService.addBookToCart(isbnNumber);
    }

    @DeleteMapping("/{isbnNumber}")
    public ResponseEntity<Cart> removeBookFromCart(@PathVariable String isbnNumber) {
        return cartService.removeBookFromCart(isbnNumber);
    }
}

And in the CartService class, you would add a method to handle removing a book from the cart:

java

@Service
@SessionScope
public class CartService {

    private final BookRepository bookRepository;
    private Cart cart;

    public CartService(BookRepository bookRepository) {
        this.bookRepository = bookRepository;
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }

    // Existing methods...

    public ResponseEntity<Cart> removeBookFromCart(String isbnNumber) {
        Iterator<Book> iterator = cart.getBookList().iterator();
        while (iterator.hasNext()) {
            Book book = iterator.next();
            if (book.getIsbnNumber().equals(isbnNumber)) {
                iterator.remove();
                cart.setUpdatedAt(Date.from(Instant.now()));
                return ResponseEntity.ok(cart);
            }
        }
        return ResponseEntity.notFound().build();
    }

    // Existing methods...
}

This removeBookFromCart method takes an ISBN number, iterates over the books in the cart, and removes the book with the matching ISBN number. If the book is found and removed successfully, it returns the updated cart; otherwise, it returns a not found response.

### Gemini

In [ ]:
Absolutely, here's how you can add an endpoint to your Spring Boot application to allow users to delete a book from their cart based on the ISBN number:

1. Update CartController:
Java

@RestController
@RequestMapping("/carts")
public class CartController {

    private final CartService cartService;

    public CartController(CartService cartService) {
        this.cartService = cartService;
    }

    @GetMapping
    public Cart getCartContent() {
        return cartService.getCart();
    }

    @PostMapping("/{isbnNumber}")
    public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
        return cartService.addBookToCart(isbnNumber);
    }

    @DeleteMapping("/{isbnNumber}") // Update to DELETE method
    public ResponseEntity<Cart> deleteBookFromCart(@PathVariable String isbnNumber) {
        return cartService.deleteBookFromCart(isbnNumber); // Call new CartService method
    }
}

Używaj kodu z rozwagą.

2. Update CartService:
Java

@Service
@SessionScope
public class CartService {

    private final BookRepository bookRepository;
    private Cart cart;

    public CartService(BookRepository bookRepository) {
        this.bookRepository = bookRepository;
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }

    public Cart getCart() {
        return cart;
    }

    public ResponseEntity<Cart> addBookToCart(String isbnNumber) {
        Mono<Book> bookMono = bookRepository.findById(isbnNumber);
        Optional<Book> book = bookMono.blockOptional();

        if (book.isPresent()) {
            cart.getBookList().add(book.get());
            cart.setUpdatedAt(Date.from(Instant.now()));
            return ResponseEntity.ok(cart);
        }

        return ResponseEntity.notFound().build();
    }

    public ResponseEntity<Cart> deleteBookFromCart(String isbnNumber) {
        boolean bookRemoved = cart.getBookList().removeIf(book -> book.getIsbnNumber().equals(isbnNumber));

        if (bookRemoved) {
            cart.setUpdatedAt(Date.from(Instant.now()));
            return ResponseEntity.ok(cart);
        }

        return ResponseEntity.notFound().build();
    }

    public void clearCart() {
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }
}

Używaj kodu z rozwagą.

Explanation:

    We've added a new method deleteBookFromCart to CartService.
    The deleteBookFromCart method iterates through the bookList using removeIf.
    removeIf removes elements from the list for which the provided condition is true.
    The condition here checks if the book's isbnNumber matches the provided isbnNumber.
    If a book is removed (bookRemoved is true), the cart's updatedAt is updated, and a successful response with the updated cart is returned.
    If no book is found with the matching isbnNumber, a not-found response is returned.

Additional Notes:

    Consider implementing error handling for potential exceptions during the deletion process.

Źródła

    github.com/DKharchenko25/JavaEnterprise_homework_3
    github.com/64byte/Web-Project 

In [ ]:
I've incorporated the best aspects of both responses A and B, addressing their potential shortcomings and considering expert feedback:

1. Update CartController:
Java

@RestController
@RequestMapping("/carts")
public class CartController {

    private final CartService cartService;

    public CartController(CartService cartService) {
        this.cartService = cartService;
    }

    @GetMapping
    public Cart getCartContent() {
        return cartService.getCart();
    }

    @PostMapping("/{isbnNumber}")
    public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
        return cartService.addBookToCart(isbnNumber);
    }

    @DeleteMapping("/{isbnNumber}") // Use DELETE method for deleting resources
    public ResponseEntity<Cart> deleteBookFromCart(@PathVariable String isbnNumber) {
        return cartService.deleteBookFromCart(isbnNumber);
    }
}

Używaj kodu z rozwagą.

2. Update CartService:
Java

@Service
@SessionScope
public class CartService {

    private final BookRepository bookRepository;
    private Cart cart;

    public CartService(BookRepository bookRepository) {
        this.bookRepository = bookRepository;
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }

    public Cart getCart() {
        return cart;
    }

    public ResponseEntity<Cart> addBookToCart(String isbnNumber) {
        // ... (existing logic for adding book to cart)
    }

    public ResponseEntity<Cart> deleteBookFromCart(String isbnNumber) {
        int removedBooks = cart.getBookList().stream()
                .filter(book -> book.getIsbnNumber().equals(isbnNumber))
                .mapToInt(book -> cart.getBookList().remove(book) ? 1 : 0)
                .sum();

        if (removedBooks > 0) {
            cart.setUpdatedAt(Date.from(Instant.now()));
            return ResponseEntity.ok(cart);
        }

        return ResponseEntity.notFound().build();
    }

    public void clearCart() {
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }
}

Używaj kodu z rozwagą.

Explanation:

    Clear and Consistent Naming: We've ensured the endpoint uses the DELETE method for consistency with RESTful conventions.
    Concise Stream-Based Removal: The deleteBookFromCart method leverages Java Streams for a more functional and concise approach to book removal.
    Optimized Removal: The filter operation ensures only matching books are considered for removal. The mapToInt operation efficiently counts the number of books removed while also removing them from the list.
    Efficient Counting: We use mapToInt instead of count() to avoid unnecessary object creation, improving performance.
    Removal Handling: The code checks if removedBooks is greater than 0 to confirm successful removal before returning a successful response.
    Error Handling: We return a not-found response (404) if no book with the matching ISBN is found in the cart.

This improved solution combines the strengths of both responses, providing a streamlined, efficient, and well-explained implementation for deleting books from the cart based on ISBN.

In [ ]:
Here's the improved code with explanations for adding an endpoint to delete a book from the cart based on its ISBN number:

1. Update CartController:
Java

@RestController
@RequestMapping("/carts")
public class CartController {

    private final CartService cartService;

    public CartController(CartService cartService) {
        this.cartService = cartService;
    }

    @GetMapping
    public Cart getCartContent() {
        return cartService.getCart();
    }

    @PostMapping("/{isbnNumber}")
    public ResponseEntity<Cart> addBookToCart(@PathVariable String isbnNumber) {
        return cartService.addBookToCart(isbnNumber);
    }

    @DeleteMapping("/{isbnNumber}") // Use DELETE for removing resources
    public ResponseEntity<Cart> deleteBookFromCart(@PathVariable String isbnNumber) {
        return cartService.deleteBookFromCart(isbnNumber);
    }
}

Używaj kodu z rozwagą.

    We've updated the deleteBookFromCart endpoint to use the DELETE HTTP method, which is the standard for deleting resources in RESTful APIs.

2. Update CartService:
Java

@Service
@SessionScope
public class CartService {

    private final BookRepository bookRepository;
    private Cart cart;

    public CartService(BookRepository bookRepository) {
        this.bookRepository = bookRepository;
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }

    public Cart getCart() {
        return cart;
    }

    public ResponseEntity<Cart> addBookToCart(String isbnNumber) {
        // ... (existing logic for adding book to cart)
    }

    public ResponseEntity<Cart> deleteBookFromCart(String isbnNumber) {
        boolean bookRemoved = cart.getBookList().removeIf(book -> book.getIsbnNumber().equals(isbnNumber));

        if (bookRemoved) {
            cart.setUpdatedAt(Date.from(Instant.now()));
            return ResponseEntity.ok(cart);
        }

        return ResponseEntity.notFound().build();
    }

    public void clearCart() {
        this.cart = new Cart(Date.from(Instant.now()), Date.from(Instant.now()), new ArrayList<>());
    }
}

Używaj kodu z rozwagą.

    The deleteBookFromCart method uses the removeIf method on the bookList.
    removeIf removes elements from the list based on a provided condition.
    The condition here checks if the isbnNumber of a book in the list matches the provided isbnNumber.
    If a book is removed (bookRemoved is true), the cart's updatedAt is updated and a successful response with the updated cart is returned.
    If no book with the matching isbnNumber is found, a not-found response (404) is returned.

Explanation:

    This approach is concise and efficient.
    removeIf avoids the need for explicit looping and simplifies removal logic.
    The code clearly indicates success or failure for deletion attempts.

This implementation effectively allows users to delete books from their cart based on their ISBN numbers while adhering to RESTful conventions.

In [ ]:
I understand you'd like to revisit the previous topic. No problem! Here's a breakdown of the code for adding an endpoint to delete a book from the cart based on its ISBN number:

1. Update in CartController:

    We modified the existing CartController class by adding a new endpoint:

Java

@DeleteMapping("/{isbnNumber}") // Use DELETE for removing resources
public ResponseEntity<Cart> deleteBookFromCart(@PathVariable String isbnNumber) {
    return cartService.deleteBookFromCart(isbnNumber);
}

Używaj kodu z rozwagą.

    This endpoint uses the DELETE HTTP method, aligning with RESTful API conventions for deleting resources. It takes an isbnNumber path variable and delegates the deletion logic to the CartService.

2. Update in CartService:

    The changes were made in the CartService class:

Java

public ResponseEntity<Cart> deleteBookFromCart(String isbnNumber) {
    boolean bookRemoved = cart.getBookList().removeIf(book -> book.getIsbnNumber().equals(isbnNumber));

    if (bookRemoved) {
        cart.setUpdatedAt(Date.from(Instant.now()));
        return ResponseEntity.ok(cart);
    }

    return ResponseEntity.notFound().build();
}

Używaj kodu z rozwagą.

    The deleteBookFromCart method utilizes the removeIf function on the cart's bookList.
    removeIf iterates through the list and removes elements that satisfy a provided condition.
    The condition here checks if the isbnNumber of a book in the list matches the provided isbnNumber.
    If a book is removed (bookRemoved is true):
        The cart's updatedAt is updated to reflect the change.
        A successful response (status code 200) with the updated cart is returned.
    If no book with the matching isbnNumber is found, a not-found response (status code 404) is returned.

Key Points:

    This approach is efficient and avoids unnecessary looping.
    removeIf simplifies the logic for removing elements based on a condition.
    The code clearly handles successful and unsuccessful deletion scenarios.

Additional Considerations:

    You could consider implementing error handling for potential exceptions during the deletion process.

I hope this explanation clarifies the code and its functionality! Feel free to ask if you have any further questions.